# Projeto 2: Como pegar dados em sites e automatizar a criação da carteira do nosso modelo de investimento.

### Desafio:

* Construir um código que vá no fundamentus.com e busque dados de todos os indicadores da bolsa brasileira e gere a carteira selecionando as maiores ev_ebit e roic.    


### Passo a passo:

   **Passo 1** - Definir um navegador que você irá utilizar para navegar com o Python.

   **Passo 2** - Importar os módulos e bibliotecas.
   
   **Passo 3** - Entender como sites e requisições na internet funcionam.
   
   **Passo 4** - Conhecer e mapear o processo de coleta de dados no site do Fundamentus. 
   
   **Passo 5** - Ler a tabela de dados.
   
   **Passo 6** - Construir a tabela final.

# Passo 1: Escolher o navegador.

No nosso caso, utilizaremos o Google Chrome. 

In [5]:
#pip install webdriver-manager
#!pip install selenium

# Passo 2: Importar as bibliotecas.

In [6]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd

# Passo 3: Entender como sites e requisições na internet funcionam.

In [7]:
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

In [8]:
url = 'https://www.fundamentus.com.br/resultado.php'

driver.get(url)

# Passo 4: Mapear o processo de coleta de dados no site do Fundamentus.

https://www.fundamentus.com.br/index.php

# Passo 5 - Ler a tabela de dados.

In [9]:
local_tabela = '/html/body/div[1]/div[2]/table'

elemento = driver.find_element("xpath", local_tabela)

html_tabela = elemento.get_attribute('outerHTML')

tabela = pd.read_html(str(html_tabela), thousands = '.', decimal = ',')[0]

# Passo 6 - Construir a tabela final.

In [10]:
tabela = tabela.set_index("Papel")
tabela = tabela[['Cotação', 'EV/EBIT', 'ROIC', 'Liq.2meses']]

tabela

,Cotação,EV/EBIT,ROIC,Liq.2meses
Papel,,,,
POPR4,10.17,0.00,"15,25%",0.0
CLAN3,0.00,0.00,"0,00%",0.0
PORP4,2.40,0.00,"0,00%",0.0
MNSA3,0.42,0.00,"-13,50%",0.0
PMET3,0.00,0.00,"0,00%",0.0
...,...,...,...,...
CSNA3,11.96,7.59,"8,82%",80254500.0
UBBR11,14.75,0.00,"0,00%",0.0
UBBR3,18.00,0.00,"0,00%",0.0


In [11]:
tabela['ROIC'] = tabela['ROIC'].str.replace("%", "")
tabela['ROIC'] = tabela['ROIC'].str.replace(".", "")
tabela['ROIC'] = tabela['ROIC'].str.replace(",", ".")
tabela['ROIC'] = tabela['ROIC'].astype(float)

In [12]:
tabela = tabela[tabela['Liq.2meses'] > 1000000]

In [13]:
tabela = tabela[tabela['EV/EBIT'] > 0]
tabela = tabela[tabela['ROIC'] > 0]

In [14]:
tabela['ranking_ev_ebit'] = tabela['EV/EBIT'].rank(ascending = True)
tabela['ranking_roic'] = tabela['ROIC'].rank(ascending = False)
tabela['ranking_total'] = tabela['ranking_ev_ebit'] + tabela['ranking_roic']

tabela = tabela.sort_values('ranking_total')

tabela.head(10)

,Cotação,EV/EBIT,ROIC,Liq.2meses,ranking_ev_ebit,ranking_roic,ranking_total
Papel,,,,,,,
PSSA3,30.10,0.27,84.27,4.201070e+07,1.0,1.0,2.0
WIZC3,5.85,1.60,27.47,2.021430e+06,2.0,9.0,11.0
PLPL3,11.41,4.01,40.24,1.017460e+07,12.0,2.0,14.0
CMIN3,5.02,4.31,32.77,4.849470e+07,15.0,5.0,20.0
PETR4,37.68,3.21,23.30,1.185670e+09,6.0,15.5,21.5
PETR3,40.73,3.39,23.30,3.965790e+08,7.0,15.5,22.5
VLID3,17.01,3.47,21.82,6.406680e+06,8.0,18.0,26.0
VALE3,60.57,3.99,20.04,1.083270e+09,11.0,22.0,33.0
LREN3,13.18,4.11,18.65,1.911640e+08,13.0,25.0,38.0
